In [ ]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install tqdm
%pip install timezonefinder
%pip install pickle
%pip install statsmodels
%pip install warnings
%pip install pytz
%pip install retry_requests
%pip install requests_cache
%pip install openmeteo_requests
%pip install plotly
%pip install python-microgrid
%pip install scikit-learn
%pip install torch

# %pip install joblib

In [15]:
from typing import List
import pytz
import warnings

In [3]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             mean_squared_log_error, median_absolute_error,
                             r2_score)

from tqdm import tqdm
from timezonefinder import TimezoneFinder

In [4]:
from utils import ReporterPlotMaker

# Constants

In [5]:
DATA_PATH = "data"
IRRADIANCE_PATH = DATA_PATH + "/psh_irradiance_10m.csv"

TIMESTAMP_COLUMN_NAME = "timestamp"
IRRADIANCE_COLUMN_NAME = "solar_irradiance"
USECOLS = [TIMESTAMP_COLUMN_NAME, IRRADIANCE_COLUMN_NAME]

FORECAST_COLUMN_NAME = "solar_irradiance_forecast"

In [6]:
LATITUDE = 18.852
LONGITUDE = 98.994
TIMEZONE = TimezoneFinder().certain_timezone_at(lat=LATITUDE, lng=LONGITUDE)

In [7]:
SAMPLE_STARTING_TIMESTAMP = pd.Timestamp(year=2024, month=3, day=1, tz=TIMEZONE)
SAMPLE_ENDING_TIMESTAMP = pd.Timestamp(year=2024, month=3, day=3, tz=TIMEZONE)

TEST_SPLIT_TIMESTAMP = pd.Timestamp(year=2025, month=2, day=16, tz=TIMEZONE)

In [8]:
IRRADIANCE_TIMESERIES_RESOLUTION = pd.Timedelta(10, "m")

# Utils

In [10]:
def manage_timestamp_column(df, timestamp_column_name, timezone):
    df[timestamp_column_name] = pd.to_datetime(df[timestamp_column_name], utc=False, )
    df[timestamp_column_name] = df[timestamp_column_name].dt.tz_convert(timezone)
    df = df.sort_values(by=timestamp_column_name)
    df = df.set_index(timestamp_column_name) # .asfreq(data_resolution_str)
    return df

In [11]:
def plot_plotly(df, 
                primary_data_columns_names, 
                secondary_data_columns_names,
                timestamp_column_name,
                title):
    reporter = ReporterPlotMaker()
    reporter.read_data(df)
    reporter.prepare_figure(primary_data_columns_names=primary_data_columns_names,
                            secondary_data_columns_names=secondary_data_columns_names,
                            title=title,
                            x_axis_column=timestamp_column_name)
    figure = reporter.plot()
    figure.show()

In [9]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true[y_true!=0] - y_pred[y_true!=0]) / y_true[y_true!=0])) * 100
    
def build_metrics_table(y_true, y_pred):
    metrics_dict = {"mean_squared_error": [mean_squared_error(y_true, y_pred)],
                    "mean_squared_log_error": [mean_squared_log_error(y_true, y_pred)],
                    "mean_absolute_error": [mean_absolute_error(y_true, y_pred)],
                    "median_absolute_error": [median_absolute_error(y_true, y_pred)],
                    "r2_score": [r2_score(y_true, y_pred)],
                    # "mean_absolute_percentage_error": [mean_absolute_percentage_error(y_true, y_pred)],
                   }
    return pd.DataFrame(metrics_dict)   

In [34]:
from abc import ABC, abstractmethod

class Forecaster(ABC):

    @property
    @abstractmethod
    def name(self) -> str:
        ...

    @abstractmethod
    def forecast(self):
        ...

    @abstractmethod
    def test_loop(self, test_ts: pd.Series) -> List:
        ...

In [40]:
class Tester:

    def __init__(self, forecaster: Forecaster, test_ts: pd.Series):
        self.forecaster = forecaster
        self.test_ts = test_ts
        self.forecasts_column_name = self.test_ts.name + "forecast" + self.forecaster.name
        
    def test(self):
        forecasts_list = self.forecaster.test_loop(self.test_ts)
        self.result_df = self.test_ts.to_frame()
        self.result_df[self.forecasts_column_name] = forecasts_list

    def plot_result(self) -> None:
        plot_plotly(self.result_df.reset_index(),
                    [self.test_ts.name, self.forecasts_column_name],
                    [],
                    TIMESTAMP_COLUMN_NAME,
                    "Test Results")

    def save_result(self) -> None:
        self.result_df.to_csv(f"{self.forecasts_column_name}.csv", index=True)

    def load_result(self, path: str) -> None:
        self.result_df = pd.read_csv(path)
        manage_timestamp_column(self.result_df, TIMESTAMP_COLUMN_NAME, TIMEZONE)

In [41]:
from models.open_meteo_irradiance_forecaster import OpenMeteoIrradianceForecaster, OpenMeteoIrradianceForecasterConfig

class OpenMeteoForecaster(Forecaster):

    def __init__(self) -> None:
        self._name = "open_meteo"
        forecaster_config = OpenMeteoIrradianceForecasterConfig(latitude=LATITUDE,
                                                        longitude=LONGITUDE,
                                                        value_column=IRRADIANCE_COLUMN_NAME,
                                                        timestamp_column=TIMESTAMP_COLUMN_NAME,
                                                        open_meteo_requests_cache_expiration_period=3600)
        forecaster = OpenMeteoIrradianceForecaster(forecaster_config)
        forecast_size = 1

    @property
    def name(self) -> str:
        return self._name

    def forecast(self):
        ...

    def test_loop(self, test_ts: pd.Series) -> List:
        openmeteo_forecasts_list = []

        row = 0
        
        for index, value in tqdm(irradiance_test_ts.items()):
            
            # print(f"Index : {index}, Value : {value}")
            
            index_floored = index.floor(forecaster.resolution)
            
            # print(f"{index_floored=}")
            
            forecast = forecaster.forecast(index_floored, forecast_size)
            
            # print(f"Forecast : {forecast}")
            
            # print(f"Forecast Value : {forecast[index_floored]}")
            # print()
            
            openmeteo_forecasts_list.append(forecast[index_floored])
            
            row += 1
        
            if row > 100_000:
                break

In [42]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

class SarimaxForecaster(Forecaster):

    def __init__(self, version_code: str) -> None:
        self._name = "sarimax_v" + version_code

    @property
    def name(self) -> str:
        return self._name

    def forecast(self):
        ...

    def test_loop(self, test_ts: pd.Series) -> List:
        ...

In [ ]:
class ModelsComparator:

    def __init__(self, testers: List[Tester]):
        self.results = self.form_results_table(testers)
        self.metrics_table = self.form_metrics_table()

    def form_results_table(self):
        ...

    def form_metrics_table(self):
        results_sequential_df = results_sequential_df.clip(lower=0)
        metrics_table_openmeteo = build_metrics_table(y_true=results_sequential_df[IRRADIANCE_COLUMN_NAME].values,
                                                      y_pred=results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_openmeteo"].values)
        metrics_table_sarimax_v1 = build_metrics_table(y_true=results_sequential_df[IRRADIANCE_COLUMN_NAME].values,
                                                       y_pred=results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_v1"].values)
        metrics_table_sarimax_v2 = build_metrics_table(y_true=results_sequential_df[IRRADIANCE_COLUMN_NAME].values,
                                                       y_pred=results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_v2"].values)
        metrics_table = pd.concat([metrics_table_openmeteo,
                                   metrics_table_sarimax_v1,
                                   metrics_table_sarimax_v2])
        
        metrics_table.index = ["openmeteo", "sarimax_v1", "sarimax_v2"]
    
        return metrics_table

    def plot_results(self):
        plot_plotly(results_sequential_df.reset_index(),
                [IRRADIANCE_COLUMN_NAME, 
                 IRRADIANCE_COLUMN_NAME + "_forecast_v1", 
                 IRRADIANCE_COLUMN_NAME + "_forecast_v2",
                 IRRADIANCE_COLUMN_NAME + "_forecast_openmeteo"],
                [],
                TIMESTAMP_COLUMN_NAME,
                f"Irradiance Forecast Sequential")
        

# Load Data

In [19]:
irradiance_df = pd.read_csv(IRRADIANCE_PATH, 
                            parse_dates=[TIMESTAMP_COLUMN_NAME],
                            usecols=USECOLS)
irradiance_df = manage_timestamp_column(irradiance_df, TIMESTAMP_COLUMN_NAME, TIMEZONE)

# Visualize Sample

In [ ]:
plot_plotly(irradiance_df[SAMPLE_STARTING_TIMESTAMP : SAMPLE_ENDING_TIMESTAMP].reset_index(),
            [IRRADIANCE_COLUMN_NAME],
            [],
            TIMESTAMP_COLUMN_NAME,
            "Irradiance")

# Split and get test Series

In [21]:
irradiance_test_df = irradiance_df[TEST_SPLIT_TIMESTAMP : ]

In [22]:
irradiance_train_df = irradiance_df[: TEST_SPLIT_TIMESTAMP]

In [23]:
len(irradiance_test_df) / len(irradiance_train_df)

0.1922030670883063

In [24]:
irradiance_test_ts = irradiance_test_df.squeeze()

# Test and Evaluation

In [43]:
open_meteo_forecaster = OpenMeteoForecaster() # baseline. currently in production
sarimax_v1_forecaster = SarimaxForecaster(version_code="1") # resolution: 10 min; history: 24h; forecast: 3h
sarimax_v2_forecaster = SarimaxForecaster(version_code="2") # resolution: 10 min; history: 48h; forecast: 1h
sarimax_v3_forecaster = SarimaxForecaster(version_code="3") # resolution: 1 hour; history: 48h; forecast: 1h

In [44]:
open_meteo_tester = Tester(open_meteo_forecaster, irradiance_test_ts)
sarimax_v1_tester = Tester(sarimax_v1_forecaster, irradiance_test_ts)
sarimax_v2_tester = Tester(sarimax_v2_forecaster, irradiance_test_ts)
sarimax_v3_tester = Tester(sarimax_v3_forecaster, irradiance_test_ts)

In [46]:
# load
open_meteo_tester.load_result("openmeteo_irradiance_result_df.csv")
sarimax_v1_tester.load_result("sarimax_10min_irradiance_result_df.csv")
sarimax_v2_tester.load_result("sarimax_10min_modified_irradiance_result_df.csv")
# sarimax_v3_tester

In [ ]:
open_meteo_tester.result_df

In [ ]:
sarimax_v1_tester.result_df

In [ ]:
sarimax_v2_tester.result_df

In [ ]:
comparator = ModelsComparator([open_meteo_tester,
                               sarimax_v1_tester,
                               sarimax_v2_tester,
                               sarimax_v3_tester])

In [ ]:
comparator.metrics_table

In [ ]:
comparator.plot_results

# SARIMAX model

In [29]:
train_series = irradiance_train_df[-7*24*6 : ] # last 2 days of train dataset

In [30]:
train_series = train_series.resample('1h').mean()

In [31]:
# old 10 min
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     model = SARIMAX(train_series, 
#                     order=(2, 0, 2), 
#                     seasonal_order=(1, 1, 1, 144),
#                     enforce_stationarity=False, 
#                     enforce_invertibility=False)

# new 10 min
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     model = SARIMAX(train_series, 
#                     order=(1, 0, 1), 
#                     seasonal_order=(1, 0, 1, 144),
#                     enforce_stationarity=False, 
#                     enforce_invertibility=False)

# new 1 hour
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = SARIMAX(train_series, 
                    order=(1, 0, 1), 
                    seasonal_order=(1, 0, 1, 24))

In [32]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model_fited = model.fit()

In [33]:
irradiance_test_ts_hourly = irradiance_test_ts.resample('1h').mean()

In [35]:
one_hour = pd.Timedelta(1, "h")
steps_in_hour = 1 # one_hour // IRRADIANCE_TIMESERIES_RESOLUTION

In [36]:
forecast_horizon = 1 * steps_in_hour
history_size = 48 * steps_in_hour

In [37]:
old_forecast_horizon = 18

In [36]:
forecasts_list = []
forecasts_timestamps_list = []

In [37]:
len(irradiance_test_ts_hourly)

1899

In [ ]:
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     for step in tqdm(range(history_size, len(irradiance_test_ts_hourly))):
#         current_timestamp = irradiance_test_ts_hourly.index[step]
#         actual_value = irradiance_test_ts_hourly.values[step]
#         history = irradiance_test_ts_hourly[step - history_size : step]
#         assert len(history) == history_size

#         model_fited = model_fited.apply(history)
#         forecast = model_fited.forecast(forecast_horizon)

#         forecasts_list.append(forecast.values)
#         forecasts_timestamps_list.append(forecast.index)

In [159]:
irradiance_result_df = irradiance_test_ts_hourly[history_size : ].to_frame()

In [160]:
forecasts_np = np.array(forecasts_list)

In [161]:
for forecast_step in range(forecast_horizon):
    irradiance_result_df[f"{forecast_step}_" + FORECAST_COLUMN_NAME] = forecasts_np[:, forecast_step]

In [ ]:
irradiance_result_df = irradiance_test_ts[(history_size * (one_hour // IRRADIANCE_TIMESERIES_RESOLUTION)) : ].to_frame()

In [165]:
irradiance_result_df = irradiance_result_df.resample('10min').ffill()

In [ ]:
irradiance_test_ts[irradiance_result_df.index[0]:].plot()

In [ ]:
len(irradiance_result_df.filnan())

In [ ]:
irradiance_result_df["solar_irradiance"] = irradiance_test_ts[irradiance_result_df.index[0]:]

In [ ]:
plot_plotly(irradiance_result_df.reset_index(),
            [IRRADIANCE_COLUMN_NAME] + [f"{i}_" + FORECAST_COLUMN_NAME for i in range(forecast_horizon)],
            [],
            TIMESTAMP_COLUMN_NAME,
            "Irradiance Forecast")

In [59]:
irradiance_result_df.to_csv("irradiance_modified_10min_result_df.csv", index=True)

In [30]:
openmeteo_irradiance_result_df = pd.read_csv("openmeteo_irradiance_result_df.csv")
openmeteo_irradiance_result_df = manage_timestamp_column(openmeteo_irradiance_result_df, TIMESTAMP_COLUMN_NAME, TIMEZONE)

# SARIMAX evaluation

In [31]:
sarimax_v1_result_df = pd.read_csv("sarimax_10min_irradiance_result_df.csv")
sarimax_v1_result_df = manage_timestamp_column(sarimax_v1_result_df, TIMESTAMP_COLUMN_NAME, TIMEZONE)

In [32]:
sarimax_v2_result_df = pd.read_csv("sarimax_10min_modified_irradiance_result_df.csv")
sarimax_v2_result_df = manage_timestamp_column(sarimax_v2_result_df, TIMESTAMP_COLUMN_NAME, TIMEZONE)

In [33]:
sarimax_v1_result_df = sarimax_v1_result_df[sarimax_v2_result_df.index[0]:]

In [ ]:
v1_result_sequential = []
v2_result_sequential = []
for forecast_timestamp in range(0, 
                                len(sarimax_v1_result_df)-(len(sarimax_v1_result_df) % forecast_horizon), 
                                forecast_horizon):
    
    v1_result_sequential.append(sarimax_v1_result_df.drop(columns=[IRRADIANCE_COLUMN_NAME] + [f"{i}_" + FORECAST_COLUMN_NAME for i in range(forecast_horizon, 
                                                                                                                                    old_forecast_horizon)]).iloc[forecast_timestamp].values)
    v2_result_sequential.append(sarimax_v2_result_df.drop(columns=[IRRADIANCE_COLUMN_NAME]).iloc[forecast_timestamp].values)
    
v1_result_sequential = np.concatenate(v1_result_sequential)
v2_result_sequential = np.concatenate(v2_result_sequential)

In [75]:
results_sequential_df = sarimax_v1_result_df[IRRADIANCE_COLUMN_NAME].to_frame()[:-(len(sarimax_v1_result_df) % forecast_horizon)]

In [77]:
results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_v1"] = v1_result_sequential
results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_v2"] = v2_result_sequential

In [106]:
results_sequential_df = results_sequential_df[openmeteo_irradiance_result_df.index[0]:]

In [ ]:
results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_openmeteo"] = openmeteo_irradiance_result_df["solar_irradiance_forecast"].values[:-1]

In [ ]:
results_sequential_df

In [ ]:
plot_plotly(results_sequential_df.reset_index(),
                [IRRADIANCE_COLUMN_NAME, 
                 IRRADIANCE_COLUMN_NAME + "_forecast_v1", 
                 IRRADIANCE_COLUMN_NAME + "_forecast_v2",
                 IRRADIANCE_COLUMN_NAME + "_forecast_openmeteo"],
                [],
                TIMESTAMP_COLUMN_NAME,
                f"Irradiance Forecast Sequential")

In [131]:
results_sequential_df = results_sequential_df.clip(lower=0)

In [132]:
metrics_table_openmeteo = build_metrics_table(y_true=results_sequential_df[IRRADIANCE_COLUMN_NAME].values,
                                              y_pred=results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_openmeteo"].values)

In [133]:
metrics_table_sarimax_v1 = build_metrics_table(y_true=results_sequential_df[IRRADIANCE_COLUMN_NAME].values,
                                               y_pred=results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_v1"].values)

In [134]:
metrics_table_sarimax_v2 = build_metrics_table(y_true=results_sequential_df[IRRADIANCE_COLUMN_NAME].values,
                                               y_pred=results_sequential_df[IRRADIANCE_COLUMN_NAME + "_forecast_v2"].values)

In [135]:
metrics_table = pd.concat([metrics_table_openmeteo,
                           metrics_table_sarimax_v1,
                           metrics_table_sarimax_v2])

metrics_table.index = ["openmeteo", "sarimax_v1", "sarimax_v2"]

In [ ]:
metrics_table